# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [79]:
import pandas as pd

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_recall_fscore_support, multilabel_confusion_matrix
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /home/sumit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sumit/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sumit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [84]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
#engine.table_names()
df = pd.read_sql_table('InsertTableName',engine)
X = df['message']
y = df[df.columns[4:]]

In [91]:
for col in y.columns:
    print(col+":"+ str(y[col].sum()))

related:19906
request:4474
offer:118
aid_related:10860
medical_help:2084
medical_products:1313
search_and_rescue:724
security:471
military:860
water:1672
food:2923
shelter:2314
clothing:405
money:604
missing_people:298
refugees:875
death:1194
other_aid:3446
infrastructure_related:1705
transport:1201
buildings:1333
electricity:532
tools:159
hospitals:283
shops:120
aid_centers:309
other_infrastructure:1151
weather_related:7297
floods:2155
storm:2443
fire:282
earthquake:2455
cold:530
other_weather:1376
direct_report:5075


### 2. Write a tokenization function to process your text data

In [86]:
def tokenize(text):
    #remove urls
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_regex, text)
    
    for url in urls:
        text = text.replace(url, 'urlplaceholder')
    
    #Normalize text
    text = re.sub(r'[^a-zA-Z]', ' ', text.lower())
    
    #Tokenize text
    words = word_tokenize(text)
    
    #Get Rid of Stop Words
    words = [w for w in words if w not in stopwords.words('english')]
    
    #lemmatize words 
    wnl = WordNetLemmatizer()
    words = [wnl.lemmatize(w) for w in words]
    
    #lemmatize words (verbs)
    words = [wnl.lemmatize(w, pos='v') for w in words]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [87]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('classifier',MultiOutputClassifier(RandomForestClassifier(), n_jobs=4))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=43, test_size=0.20)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function toke...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                    

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [97]:
y_pred = pipeline.predict(X_test)

In [52]:
y_pred_df = pd.DataFrame(y_pred, columns=y.columns)
y_pred_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
# import numpy as np
# for col in y.columns:
#     print("classification report for {}" .format(col))
#     #print(classification_report(np.array(y_test[col]), np.array(y_pred_df[col])))

print(classification_report(y_test, y_pred, target_names=y.columns.values))    

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      3956
               request       0.85      0.47      0.60       894
                 offer       0.00      0.00      0.00        19
           aid_related       0.75      0.70      0.72      2141
          medical_help       0.53      0.07      0.12       414
      medical_products       0.77      0.08      0.14       250
     search_and_rescue       0.78      0.05      0.10       134
              security       1.00      0.01      0.02        93
              military       0.67      0.07      0.13       169
                 water       0.91      0.38      0.53       339
                  food       0.84      0.60      0.70       581
               shelter       0.82      0.37      0.51       437
              clothing       0.88      0.09      0.16        80
                 money       0.40      0.02      0.04       107
        missing_people       0.50      

/home/sumit/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sumit/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sumit/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [64]:
multilabel_confusion_matrix(y_test, y_pred)

array([[[ 758, 1115],
        [ 316, 5620]],

       [[6368,  122],
        [ 687,  632]],

       [[7781,    0],
        [  28,    0]],

       [[3862,  701],
        [ 996, 2250]],

       [[7145,   49],
        [ 561,   54]],

       [[7415,   13],
        [ 354,   27]],

       [[7591,    5],
        [ 200,   13]],

       [[7673,    3],
        [ 133,    0]],

       [[7532,    7],
        [ 257,   13]],

       [[7809,    0],
        [   0,    0]],

       [[7265,   21],
        [ 335,  188]],

       [[6840,  107],
        [ 323,  539]],

       [[7080,   50],
        [ 435,  244]],

       [[7687,    4],
        [ 109,    9]],

       [[7634,    3],
        [ 167,    5]],

       [[7729,    2],
        [  76,    2]],

       [[7540,   12],
        [ 242,   15]],

       [[7427,    9],
        [ 344,   29]],

       [[6783,   14],
        [ 980,   32]],

       [[7311,    6],
        [ 492,    0]],

       [[7450,   10],
        [ 322,   27]],

       [[7427,   12],
        [ 32

### 6. Improve your model
Use grid search to find better parameters. 

In [54]:
# parameters = pipeline.get_parameters()
pipeline.get_params()


{'memory': None,
 'steps': [('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x7fb62bc944c0>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('classifier',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                         

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [57]:
cv =CountVectorizer(tokenizer=tokenize)
cv.fit_transform(X)

<26028x25768 sparse matrix of type '<class 'numpy.int64'>'
	with 343309 stored elements in Compressed Sparse Row format>

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.